**Streaming tweets from the Twitter API v1.1**

First you need have to a Twitter account. After that go to Twitter Apps.

Next, make a new application, filling in your name, description, website, agree to their terms, do the captcha, and create the application.

Once submitted successfully, you should be presented with a page where you can see your consumer key and consumer secret. Now you need an access token, so scroll down and click on "create my access token".

In [1]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
from tweepy import API, Cursor

import pandas as pd
import numpy as np

In [2]:
#Get consumer key, consumer secret, access token and access secret from your account

ckey = 'NQb8eQapFBwILFLhQ9iqRxFeF'
csecret = 'sZCLTQW6PfaygcmMfvde1BELCCxGdXoWXjL8Uv8lvLuOlGroHt'
akey = '1152874593327054848-ilOE3GXanLAFs7eZjD9pRbUJVigBfb'
asecret = '00VcFNOIDDshYNEGnkplHCOItAWxGcipozQ5hb8kMZCGg' 

In [3]:
# Giving credentials to the program
class TwitterAuthenticator():
                            
    def authenticate_twitter_app(app):
        auth = OAuthHandler(ckey, csecret)
        auth.set_access_token(akey, asecret)
        return auth
    
class TwitterClient():
    
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)
        
        self.twitter_user = twitter_user
        
    def get_twitter_client_api(self):
        return self.twitter_client
        
    def get_usertimeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id = self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id = self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list
    
    def get_hometimeline_tweets(self, num_tweets):
        hometimeline_list = []
        for tweet in Cursor(self.twitter_client.home_timeline, id = self.twitter_user).items(num_tweets):
            hometimeline_list.append(tweet)
        return hometimeline_list

In [4]:
# This class just prints the tweets received from the Stream
class TwitterStreamListener(StreamListener):
    
    def __init__(self, tweets_filename):
        self.tweets_filename = tweets_filename
        
    def on_data(self, data):
        try:
            print(data)
            with open(self.tweets_filename, a) as f:
                f.write(data)
        except BaseException as e:
            print("Error on data {}".format(e))
        return True
    
    def on_error(self, status):
        print(status)
        
# Class for streaming and processing live tweets        
class TwitterStreamer():
    
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
    # This handles Twitter authentication and connection to the Twitter API
    def stream_tweets(self, tweets_filename, track_list):
        
        listener = TwitterStreamListener(tweets_filename)
        auth = self.twitter_authenticator.authenticate_twitter_app()
        stream = Stream(auth, listener)
    
    # This filters the Twitter Stream for the provided keywords
        stream.filter(track = track_list)

In [5]:
# Functionality for analyzing and categorizing content form tweets
class TwitterAnalyzer():
    
    def tweets_to_dataframe(self, tweets):
        
        df = pd.DataFrame(data = [tweet.text for tweet in tweets], columns = ['Tweets'])
        
        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        
        return df

In [6]:
track_list = ["donal trump", "hillary clinton", "barack obama", "bernie sanders"]
tweets_filename = "tweets.txt"

# twitter_client = TwitterClient(twitter_user='pycon')
# print(twitter_client.get_usertimeline_tweets(10))

# twitter_streamer = TwitterStreamer()
# twitter_streamer.stream_tweets(tweets_filename, track_list)

In [8]:
twitter_client = TwitterClient()
api = twitter_client.get_twitter_client_api()

tweets = api.user_timeline(screen_name="realDonaldTrump", count=20)
df = TwitterAnalyzer().tweets_to_dataframe(tweets)
print(df.head(10))

                                              Tweets                   id  \
0  RT @Jim_Jordan: Democrats on the House Judicia...  1172354179358232576   
1  RT @VP: Even the Washington Post, the Democrat...  1172354049624170498   
2   RT @KellyannePolls: Wow. https://t.co/DJ2Stc2MhF  1172353428288430080   
3  RT @Trump: Proud to announce that @TrumpNewYor...  1172353230505938946   
4  RT @Jim_Jordan: Looks like we might be getting...  1172353025270276097   
5  RT @Jim_Jordan: Comey:\n-Opened Trump-Russia i...  1172352958379495424   
6  RT @RepRatcliffe: If reports are accurate that...  1172352873046429696   
7           Hello Baltimore! https://t.co/Iz2aYj7rrC  1172280991873978368   
8  In fact, my views on Venezuela, and especially...  1172198767627526151   
9  “This should have been over with after the Mue...  1172151706701959171   

   len                date              source  retweets  likes  
0  140 2019-09-13 03:39:52  Twitter for iPhone     22110      0  
1  140 2019-09-13 03